# Versuch 2 - Kalibrierung von Digitalkameras

Dominik Bartsch, Stefan Ptacek, Niklas Kaiser

In [1]:
!python -m pip install opencv-python
import cv2
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


## 1. Aufnahme und Analyse eines Grauwertkeiles
### 1.1 Einlesen der Werte aus der Kamera

In [4]:
# -------- Aufgabe1.1 -------- #

cap = cv2.VideoCapture(0)

for i in range(10):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("./data/fix" + str(i) + ".png", gray)

print("frame width: " + str(cap.get(3)))
print("frame height: " + str(cap.get(4)))
print("--------------------------------")
print("brightness: " + str(cap.get(10)))
print("contrast: " + str(cap.get(11)))
print("saturation: " + str(cap.get(12)))
print("--------------------------------")
print("gain: " + str(cap.get(14)))
print("exposure: " + str(cap.get(15)))
print("--------------------------------")
print("white balance: " + str(cap.get(17)))

cap.release()
#cv2.destroyAllWindows()

[ WARN:0@11.575] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@11.575] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.8.1) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


### 1.2 Mittelwert und die Standartabweichung

In [2]:
image = cv2.imread("./data/dark1.png")
image = image.astype('float32')

schwarz = image[0:450,0:130]
dunkelgrau = image[:,140:270]
mittelgrau = image[:,280:410]
hellgrau = image[:,420:540]
weiß = image[:,560:640]

cv2.imwrite("./data/changed/black.png", schwarz.astype('uint8'))
cv2.imwrite("./data/changed/dark1.png", dunkelgrau.astype('uint8'))
cv2.imwrite("./data/changed/dark2.png", mittelgrau.astype('uint8'))
cv2.imwrite("./data/changed/dark3.png", hellgrau.astype('uint8'))
cv2.imwrite("./data/changed/white.png", weiß.astype('uint8'))

print("\t\t| std \t\t| mean")
print("-----------------------------------------------------")
print("schwarz \t| %f \t| %f" %(np.std(schwarz),np.mean(schwarz)))
print("dunkelgrau \t| %f \t| %f" %(np.std(dunkelgrau),np.mean(dunkelgrau)))
print("mittelgrau \t| %f \t| %f" %(np.std(mittelgrau),np.mean(mittelgrau)))
print("hellgrau \t| %f \t| %f" %(np.std(hellgrau),np.mean(hellgrau)))
print("weiß \t\t| %f \t| %f" %(np.std(weiß),np.mean(weiß)))

		| std 		| mean
-----------------------------------------------------
schwarz 	| 2.641622 	| 29.286991
dunkelgrau 	| 2.657652 	| 68.133286
mittelgrau 	| 1.679518 	| 112.853996
hellgrau 	| 2.370252 	| 151.935501
weiß 		| 2.986076 	| 195.880798


## 2. Aufnahme eines Dunkelbildes

In [35]:
imgpath = "./data/black"
imgpath2 = "./data/white0.png"
imgarr = []


def pixmean(path):
    for i in range(10):
        img = cv2.imread(path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        imgarr.append(img)

    return np.mean(imgarr, axis=0).astype(np.double)

darkmean = pixmean(imgpath)
cv2.imwrite("./data/changed/darkmean.png", darkmean.astype('uint8'))

def submeandark(img):
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    return img - darkmean

new_img = submeandark(imgpath2)
cv2.imwrite("./data/changed/fixedimage.png", new_img.astype('uint8'))

True

### Mittelwert Dunkelbild:
![](data/changed/darkmean.png)

### Ausgangsbild
![](data/changed/fixedimage.png)

## 3. Aufnahme eines Weißbildes

In [62]:
def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


def fix_noise(dark_img, light_img, img):
    img = img - dark_img
    light_img = (light_img - dark_img) / np.mean(light_img)
    img = img / light_img
    return img


def bildKontrastMaximiert(img, filename):
    imgCorrected = np.zeros(shape=(len(img), len(img[0])))
    min = np.min(img)
    max = np.max(img)
    for i in range(480):
        for j in range(640):
            imgCorrected[i][j] = img[i][j] - min
            if max > min:
                imgCorrected[i][j] = imgCorrected[i][j] * (255 / (max - min))
            else:
                imgCorrected[i][j] = imgCorrected[i][j] * 255

    cv2.imwrite("data/changed/" + filename + ".png", imgCorrected.astype('uint8'))

    count = 0
    for i in range(len(img)):
        for j in range(len(img[0])):
            if img[i][j] == imgCorrected[i][j]:
                count += 1
    return imgCorrected


imagedb = get_image("./data/black") #dunkelbild
# imagedb = imagedb.astype('float32')

imagewb = get_image("./data/white") #weißbild
# imagewb = imagewb.astype('float32')


eingangsbildname = "dark1"
imageub = cv2.imread("./data/" + eingangsbildname + ".png", cv2.IMREAD_GRAYSCALE) #unbearbeitetes einlesebild
imageub = imageub.astype('float32')

darksubtract = np.subtract(imageub,imagedb) # Dunkelbild von Eingangsbild abziehen

cv2.imwrite("./data/changed/black_bearbeitet.png", darksubtract.astype('uint8'))

imgCorrected = bildKontrastMaximiert(np.subtract(imagewb, imagedb), "white_kontrastmaximiert")
meanwhite = np.divide(imgCorrected, np.mean(imgCorrected)) # Weißbild normiert mit Mittelwert zu 1
print(np.mean(imgCorrected))
cv2.imwrite("./data/changed/meanwhitebearbeitet.png", imagewb.astype('uint8'))

fin = np.zeros(shape=(len(darksubtract), len(darksubtract[0])))

for i in range(len(darksubtract)):
    for j in range(len(darksubtract[0])):
        if meanwhite[i][j] > 0:
            fin[i][j] = float(darksubtract[i][j]) / float(meanwhite[i][j])
        else:
            print("Deadpixel x/y: {}/{}".format(j, i))
            fin[i][j] = float(darksubtract[i][j])


cv2.imwrite("./data/changed/" + eingangsbildname + "_bearbeitet.png", fin.astype('uint8'))

152.6381111870247
Deadpixel x/y: 633/0


True

### Kontrastmaximiertes Weißbild:
![](./data/changed/white_kontrastmaximiert.png)

### Eingangsbild
![](./data/dark1.png)

### Korrigiertes Eingangsbild
![](./data/changed/dark1_bearbeitet.png)

## 4. Pixelfehler

In [24]:
def get_hot_pixels():
    dark_img = get_image("./data/black")
    return np.any(dark_img[:, :] > 0)


def get_stuck_pixels():
    dark_img = get_image("./data/black")
    light_img = get_image("./data/white")
    return np.any(dark_img[:, :] == light_img[:, :])


def get_dead_pixels():
    light_img = get_image("./data/white")
    return np.any(light_img[:, :] == 0)


def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


print("Are there Hotpixels? " + ("Yes" if get_hot_pixels() else "No"))
print("Are there Stuckpixels? " + ("Yes" if get_stuck_pixels() else "No"))
print("Are there Deadpixels? " + ("Yes" if get_dead_pixels() else "No"))

Are there Hotpixels? No
Are there Stuckpixels? No
Are there Deadpixels? No


In [63]:
image = cv2.imread("./data/changed/dark1_bearbeitet.png").astype('float32')

schwarz = image[0:450,0:130]
dunkelgrau = image[:,140:270]
mittelgrau = image[:,280:410]
hellgrau = image[:,420:540]
weiß = image[:,560:640]

cv2.imwrite("./data/changed/4black.png", schwarz.astype('uint8'))
cv2.imwrite("./data/changed/4dark1.png", dunkelgrau.astype('uint8'))
cv2.imwrite("./data/changed/4dark2.png", mittelgrau.astype('uint8'))
cv2.imwrite("./data/changed/4dark3.png", hellgrau.astype('uint8'))
cv2.imwrite("./data/changed/4white.png", weiß.astype('uint8'))

print("\t\t| std \t\t| mean")
print("-----------------------------------------------------")
print("schwarz \t| %f \t| %f" %(np.std(schwarz),np.mean(schwarz)))
print("dunkelgrau \t| %f \t| %f" %(np.std(dunkelgrau),np.mean(dunkelgrau)))
print("mittelgrau \t| %f \t| %f" %(np.std(mittelgrau),np.mean(mittelgrau)))
print("hellgrau \t| %f \t| %f" %(np.std(hellgrau),np.mean(hellgrau)))
print("weiß \t\t| %f \t| %f" %(np.std(weiß),np.mean(weiß)))

		| std 		| mean
-----------------------------------------------------
schwarz 	| 8.557275 	| 35.107777
dunkelgrau 	| 9.791692 	| 58.970387
mittelgrau 	| 17.372145 	| 94.900917
hellgrau 	| 34.676926 	| 160.336105
weiß 		| 87.195442 	| 124.761276


### Korrigierter Grauwertkeil
![](./data/changed/dark1_bearbeitet.png)

Todo:
- Kontrastmaximierung bei Weißbild
- Weißbildkorrektur